In [ ]:
import numpy as np
import pde

In [ ]:
print(dir(pde.pdes))

In [ ]:
CHPDE = pde.pdes.DiffusionPDE

In [ ]:
from pde import ScalarField

class MyCahnHilliardPDE(pde.pdes.CahnHilliardPDE):

    check_implementation = False

    def __init__(self, *args, affinity=5.0, **kwargs):
        super().__init__(*args, **kwargs)

        self.affinity = affinity

    def chemical_potential_f(self, u):

        return np.log(u / (1 - u)) + self.affinity * u * (1 - u)

    def mobility(self, u):
        return 1.

    def transformed_state(self, u):

        return np.ln(u / (1 - u))

    def physical_state(self, y):

        return np.exp(y) / (1 + np.exp(y))

    def du_dy(u, y):
        return np.exp(-y) / u

    def expression(self):
        raise RuntimeError("'expression' should not be called.")

    def evolution_rate(self, state: pde.ScalarField, t: float = 0) -> pde.ScalarField:

        assert isinstance(state, pde.ScalarField)

        # The Neumann condition for mu translates into a Dirichlet condition
        # for the flux.
        bc_flux = ["dirichlet", {"dirichlet": self.bc_mu[1]["neumann"]}]

        c = self.physical_state(state)

        du_dy = self.du_dy(c, state)

        c_laplace = c.laplace(bc=self.bc_c, label="evolution rate", args={"t": t})

        mu = self.chemical_potential_f(c) - self.interface_width * c_laplace

        diff_flux = self.mobility(c) * mu.gradient(bc=self.bc_mu)

        return -diff_flux.divergence(bc=bc_flux) / du_dy


## Test the implementation

Test the implementation against py-pde's CahnHilliardPDE.

In [ ]:
class ReferenceCHPDE(pde.pdes.CahnHilliardPDE):
    pass

In [ ]:
# Grid and initial data
n_x = 50

grid = pde.CartesianGrid(([0, 1],), n_x, periodic=[True, ])

x = grid.axes_coords[0]

field = pde.ScalarField(grid, 0.5 + 0.01 * np.sin(x * 2 * np.pi))

field.plot()

In [ ]:
ref_pde = pde.pdes.CahnHilliardPDE()

solver = pde.solvers.CrankNicolsonSolver(ref_pde, explicit_fraction=0.5)

result = ref_pde.solve(
    field,
    t_range=1.0,
    dt=1e-5,
    solver="scipy",
    adaptive=True,
)

result.plot()

In [ ]:
class MyDiffusionPDE(MyCahnHilliardPDE):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, interface_width=0., affinity=None, **kwargs)

    def chemical_potential_f(self, u):
        return u

    # def transformed_state(self, u):

    #     return u

    # def physical_state(self, y):

    #     return y

    # def du_dy(u, y):
    #     return 1.

In [ ]:
grid = pde.SphericalSymGrid(1., 50)

In [ ]:
field = pde.ScalarField(grid, 0.4 + 0.01 * np.random.random(50))

In [ ]:
q_dot = 1.

bc_c = ["neumann", "neumann"]
bc_mu = ["neumann", {"neumann": q_dot}]

bc_diff = [{"dirichlet": 1.}, {"dirichlet": -0.}]

eq = MyCahnHilliardPDE(affinity=5, interface_width=1., bc_mu=bc_mu, bc_c=bc_c)
# eq = pde.pdes.DiffusionPDE(bc=bc_diff)

In [ ]:
result = eq.solve(field, t_range=1.0, dt=1e-5)

In [ ]:
result.plot()

In [ ]:
result